# WebScrapping de HowLongToBeat.com

In [3]:
#Librerias
import re
import pandas as pd
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import time

In [4]:
# Funcion get para tratar las excepciones

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/59.0.3071.115 Safari/537.36'}
    try:
        with closing(get(url, stream=True, headers=head)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

## Requests HTTP

In [ ]:
#Genero una lista con todos los ids que podria llegar a tener la pagina
#ids = list(range(60000))

In [72]:
ids = list(range(30000,60000))

In [73]:
%%time
##Loopeo sobre los id de la URL
games = []


for id in ids:
    try:
        url = 'https://howlongtobeat.com/game.php?id='+str(id)
        raw_html = simple_get(url)
        html = BeautifulSoup(raw_html, 'html.parser')

        ## Titulo del juego
        titulo = html.find('div', {'class':"profile_header shadow_text"}).text.split('\n')
        titulo = titulo[1].rstrip()
            #Pasamos a dict
        titulo_dict = {'Id': id,'Titulo' : titulo}


        ## Rating
        det=html.find('div', {'class':'profile_details'})
        details = []
        for i, li in enumerate(det.select('li')):
            dicc = {i: li.text}
            details.append(li.text)
        rating = details[4][0:2]
            #Pasamos a dict
        rating_dict = {'Rating': rating}
        rating_dict


        ## Completion Times
        tiempos = html.find_all('li',class_=re.compile("short time_"))
            #Descompongo cada uno de los tiempos
        times = []
        for n in range(4):
            x = tiempos[n].select('div')
            x = str(x[0])
            x = x.replace('<div>', '')
            x = x.replace('</div>','')
            x = x.rstrip()
            times.append(x)
            #Creo un diccionario con la clave de cada tiempo
        times_dict = {'Main Story'    : times[0],
                      'Main + Extras' : times[1],
                      'Completionist' : times[2],
                      'All Styles'    : times[3]}


        ## Info del juego
        prof = html.find_all('div', {'class':'profile_info'})
            #Creo un diccionario con la info
        info_dict = {}
        for i in range(len(prof)):
            info_dict[str(prof[i].contents[1]).replace('<strong>','').replace('</strong>','').replace('\n','').replace(':','').rstrip()]\
            = prof[i].contents[2].replace('\n','').rstrip()

        ## Junto Todos los diccionarios en uno solo

        games.append({**titulo_dict, **rating_dict, **times_dict, **info_dict})
        time.sleep(2) # Esperar dos segundos entre cada loop
    except:
        pass

Error during requests to https://howlongtobeat.com/game.php?id=42116 : HTTPSConnectionPool(host='howlongtobeat.com', port=443): Max retries exceeded with url: /game.php?id=42116 (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')",),))


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Wall time: 20h 58min 25s


In [ ]:
#Primer Loop 30.000 requests 21hs 6min
#Segundo Loop 20hs 58min

In [75]:
len(games)

24599

# Agregar un loop que loopee sobre cada diccionario de la lista games y arregle las llaves, borrando las viejas

In [76]:
for i in range(len(games)):
    try:
        games[i]['Publishers']=games[i][' Publisher']
        games[i].pop(' Publisher')
    except:
        pass

In [77]:
for i in range(len(games)):
    try:
        games[i]['Publishers']=games[i]['Publisher']
        games[i].pop('Publisher')
    except:
        pass

In [78]:
for i in range(len(games)):
    try:
        games[i]['Publishers']=games[i][' Publishers']
        games[i].pop(' Publishers')
    except:
        pass

In [79]:
for i in range(len(games)):
    try:
        games[i]['Developers']=games[i][' Developer']
        games[i].pop(' Developer')
    except:
        pass

In [80]:
for i in range(len(games)):
    try:
        games[i]['Developers']=games[i]['Developer']
        games[i].pop('Developer')
    except:
        pass

In [81]:
for i in range(len(games)):
    try:
        games[i]['Developers']=games[i][' Developers']
        games[i].pop(' Developers')
    except:
        pass

In [82]:
for i in range(len(games)):
    try:
        games[i]['Genres']=games[i][' Genre']
        games[i].pop(' Genre')
    except:
        pass

In [83]:
for i in range(len(games)):
    try:
        games[i]['Genres']=games[i]['Genre']
        games[i].pop('Genre')
    except:
        pass

In [84]:
for i in range(len(games)):
    try:
        games[i]['Genres']=games[i][' Genres']
        games[i].pop(' Genres')
    except:
        pass

In [85]:
for i in range(len(games)):
    try:
        games[i]['Playable On']=games[i][' Playable On']
        games[i].pop(' Playable On')
    except:
        pass

In [86]:
# Convierto la lista en un dataframe
games_df = pd.DataFrame(games)

In [87]:
games_df.columns

Index(['All Styles', 'Completionist', 'Developers', 'EU', 'Genres', 'Id', 'JP',
       'Main + Extras', 'Main Story', 'NA', 'Playable On', 'Publishers',
       'Rating', 'Titulo', 'Type', 'Updated'],
      dtype='object')

In [88]:
#Reordeno las columnas para una mejor lectura
games_df = games_df[['Id','Titulo','Main Story','Main + Extras','Completionist','All Styles','Rating','Developers', 'Publishers', 'Playable On','Genres','Type','NA','EU','JP','Updated']]

In [89]:
games_df

,Id,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developers,Publishers,Playable On,Genres,Type,NA,EU,JP,Updated
0,30002,Executive Assault,--,--,--,--,NR,Hesketh Studios,Hesketh Studios,NaN,"Real-Time, Strategy",NaN,"July 17, 2015",NaN,NaN,1 Week Ago
1,30003,The Witcher 3: Wild Hunt - Hearts of Stone,10 Hours,14 Hours,19 Hours,14 Hours,91,CD Projekt RED,CD Projekt,"PC, PlayStation 4, Xbox One","Third-Person, Action, Adventure, Open World, R...",DLC/Expansion,"October 13, 2015","October 13, 2015",NaN,8 Hours Ago
2,30004,MadOut Ice Storm,--,--,--,--,50,Nuligine,Nuligine,NaN,Racing,NaN,"September 04, 2015","September 04, 2015",NaN,3.5 Weeks Ago
3,30005,Trap Them,--,--,--,--,NR,Juri Schupilo,Juri Schupilo,NaN,NaN,NaN,"July 10, 2015",NaN,NaN,1 Month Ago
4,30006,Ellie - Help me out...please,--,--,--,--,NR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
5,30007,Interstellar Rift,--,--,--,--,NR,Split Polygon,Split Polygon,NaN,"Open World, Simulation",NaN,"June 23, 2015",NaN,NaN,3 Months Ago
6,30008,Destiny: The Taken King,--,--,--,--,NR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
7,30009,Destiny Expansion 1: The Dark Below,--,--,--,--,NR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
8,30010,Destiny Expansion II: The House of Wolves,--,--,--,--,NR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
9,30012,Dragon Age: Inquisition - Trespasser DLC,6 Hours,8 Hours,8½ Hours,7 Hours,83,BioWare,Electronic Arts,"PC, PlayStation 4, Xbox One","Action, Role-Playing",DLC/Expansion,"September 08, 2015",NaN,NaN,1.5 Days Ago


In [90]:
#Genero un archivo CSV con los resultados del scrapping. Uso como separador pipes, ya que los juegos con mas de un Developer o
# publisher, estan separados por comas

games_df.to_csv('C:/Users/rharari/Desktop/Data Science/DB HLTB2.csv', sep='|')

In [ ]:
# Primer loop 6hs 56 min
# Segundo Loop 5hs 35 min
# Tercer loop 4hs 56min
# Cuarto loop 9hs 25 min con 20.000
# Quinto loop 3hs 59 min